# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Dumont  ->  A. Dumont  |  ['A. Dumont']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
J. Liu  ->  J. Liu  |  ['J. Liu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 60 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.09697


extracting tarball to tmp_2503.09697... done.


A. Dumont  ->  A. Dumont  |  ['A. Dumont']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']


Found 26 bibliographic references in tmp_2503.09697/MAIN_PAPER.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.09698
extracting tarball to tmp_2503.09698... done.


...

extracting tarball to tmp_2503.10180...

 done.
  0: tmp_2503.10180/highz_lens.tex, 892 lines
  1: tmp_2503.10180/natnotes.tex, 332 lines
  2: tmp_2503.10180/natbib.tex, 96 lines
Retrieving document from  https://arxiv.org/e-print/2503.10339
extracting tarball to tmp_2503.10339...

/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 3 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2503.10180/highz_lens.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


 done.


Found 73 bibliographic references in tmp_2503.10339/main.bbl.
Error retrieving bib data for liu2024: 'author'
Retrieving document from  https://arxiv.org/e-print/2503.10346
extracting tarball to tmp_2503.10346... done.
Retrieving document from  https://arxiv.org/e-print/2503.10423
extracting tarball to tmp_2503.10423... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.09697-b31b1b.svg)](https://arxiv.org/abs/2503.09697) | **WIggle Corrector Kit for NIRSpEc Data: WICKED**  |
|| <mark>A. Dumont</mark>, et al. -- incl., <mark>N. Neumayer</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *Submitted to A&A*|
|**Abstract**|            The point-spread function of the integral-field unit (IFU) mode of the JWST's NIRSpec is heavily under-sampled, creating resampling noise seen as low-frequency sinusoidal-like artifacts, or "wiggles". These artifacts in the data are not corrected in the JWST data pipeline, and significantly impact the science that can be achieved at a single-pixel level. We present WICKED (WIggle Corrector Kit for NIRSpEc Data), a tool designed to empirically remove wiggles. WICKED uses the Fast Fourier Transform to identify wiggle-affected spaxels across the data cube. Spectra are modeled with a mix of integrated aperture and annular templates, a power-law, and a second-degree polynomial. The method works across all medium- and high-resolution NIRSpec gratings: F070LP, F100LP, F170LP, and F290LP. WICKED can recover the true overall spectral shape up to a factor of 3.5x better compared to uncorrected spectra. It recovers the equivalent width of absorption lines within 5% of the true value-~3x better than uncorrected spectra and ~2x better than other methods. WICKED significantly improves kinematic measurements, recovering the line-of-sight velocity (LOSV) within 1% of the true value -- more than 100x better than uncorrected spectra at S/N ~40. As a case study, we applied WICKED to G235H/F170LP IFU data of the elliptical galaxy NGC5128, finding good agreement with previous studies. In wiggle-affected regions, the uncorrected spectrum showed stellar LOSV and velocity dispersion differences compared to the WICKED-cleaned spectrum, of ~17x and ~36x larger than the estimated uncertainties, respectively. Wiggles in NIRSpec IFU data can introduce severe biases in spectral shape, line measurements, and kinematics to values larger than the typical uncertainties. WICKED provides a robust, user-friendly solution, enabling precise single-pixel studies and maximizing JWST's potential.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.09698-b31b1b.svg)](https://arxiv.org/abs/2503.09698) | **Low 4.5 μm Dayside Emission Disfavors a Dark Bare-Rock scenario for the Hot Super-Earth TOI-431 b**  |
|| C. Monaghan, et al. -- incl., <mark>I. J. M. Crossfield</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *14 pages, 5 figures, accepted for publication in The Astronomical Journal*|
|**Abstract**|            The full range of conditions under which rocky planets can host atmospheres remains poorly understood, especially in the regime of close-in orbits around late-type stars. One way to assess the presence of atmospheres on rocky exoplanets is to measure their dayside emission as they are eclipsed by their host stars. Here, we present Spitzer observations of the 4.5 $\mu$m secondary eclipses of the rocky super-Earth TOI-431 b, whose mass and radius indicate an Earth-like bulk composition (3.07 $\pm$ 0.35 M$_{\oplus}$, 1.28 $\pm$ 0.04 R$_{\oplus}$). Exposed to more than 2000 times the irradiation of Earth, dayside temperatures of up to 2400K are expected if the planet is a dark bare-rock without a significant atmosphere. Intriguingly, despite the strong stellar insolation, we measure a secondary eclipse depth of only 33 $\pm$ 22 ppm, which corresponds to a dayside brightness temperature of $1520_{-390}^{+360}$K. This notably low eclipse depth disagrees with the dark bare-rock scenario at the 2.5$\sigma$ level, and suggests either that the planet is surrounded by an atmosphere, or that it is a bare-rock with a highly reflective surface. In the atmosphere scenario, the low dayside emission implies the efficient redistribution of heat to the nightside, or by molecular absorption in the 4-5 $\mu$m bandpass. In the bare-rock scenario, a surface composition made of a high-albedo mineral species such as ultramafic rock can lead to reduced thermal emission consistent with low eclipse depth measurement. Follow-up spectroscopic observations with the James Webb Space Telescope hold the key to constraining the nature of the planet.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.10339-b31b1b.svg)](https://arxiv.org/abs/2503.10339) | **C3PO IV: co-natal stars depleted in refractories are magnetically more active -- possible imprints of planets**  |
|| J. Yu, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *13 pages, 12 figures. Accepted for publication in MNRAS*|
|**Abstract**|            Chemical abundance anomalies in twin stars have recently been considered tell-tale signs of interactions between stars and planets. While such signals are prevalent, their nature remains a subject of debate. On one hand, exoplanet formation may induce chemical depletion in host stars by locking up refractory elements. On the other hand, exoplanet engulfment can result in chemical enrichment, both processes potentially producing similar differential signals. In this study, we aim to observationally disentangle these processes by using the Ca II infrared triplet to measure the magnetic activity of 125 co-moving star pairs with high SNR, high-resolution spectra from the Magellan, Keck, and VLT telescopes. We find that co-natal star pairs in which the two stars exhibit significant chemical abundance differences also show differences in their magnetic activity, with stars depleted in refractories being magnetically more active. Furthermore, the strength of this correlation between differential chemical abundances and differential magnetic activity increases with condensation temperature. One possible explanation is that the chemical anomaly signature may be linked to planet formation, wherein refractory elements are locked into planets, and the host stars become more active due to more efficient contraction during the pre-main-sequence phase or star-planet tidal and magnetic interactions.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.09952-b31b1b.svg)](https://arxiv.org/abs/2503.09952) | **The SRG/eROSITA All-Sky Survey : Subaru/HSC-SSP weak-lensing mass measurements for the eRASS1 Galaxy Clusters**  |
|| N. Okabe, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *27 pages, 24 figures; submitted to A&A*|
|**Abstract**|            We perform individual weak-lensing (WL) mass measurements for 78 eROSITA's first All-Sky Survey (eRASS1) clusters in the footprint of Hyper Suprime-Cam Subaru Strategic Program (HSC-SSP) S19A, without any priors of the eRASS1 X-ray quantities or assumption of the mass and concentration relation. In the sample, we find 3 clusters are mis-associated with optical counterparts and 12 clusters are poorly fitted with an NFW profile. The average mass for the 12 poor-fit clusters changes from $\sim 10^{14}h_{70}^{-1}M_\odot$ to $\sim 2\times 10^{13}h_{70}^{-1}M_\odot$ when lensing contamination from surrounding mass structures is taken into account. The scaling relations between the true mass and cluster richness and X-ray count-rate agree well with the results of the eRASS1 Western Galactic Hemisphere region based on count-rate-inferred masses, which are calibrated with the HSC-SSP, DES, and KiDS surveys. We develop a Bayesian framework for inferring the mass-concentration relation of the cluster sample, explicitly incorporating the effects of weak-lensing mass calibration in the mass-concentration parameter space. The redshift-dependent mass and concentration relation is in excellent agreement with predictions of dark-matter-only numerical simulations and previous studies using X-ray-selected clusters. Based on the two-dimensional (2D) WL analysis, the offsets between the WL-determined centers and the X-ray centroids for 36 eRASS1 clusters with high WL S/N are described by two Gaussian components. We find that the mis-centering effect with X-ray centroids is smaller than that with peaks in the galaxy maps. The projected halo ellipticity is $\langle \varepsilon \rangle=0.45$ at $M_{200}\sim 4\times10^{14}h_{70}^{-1}M_\odot$, in agreement with results of numerical simulations and previous studies of clusters more than twice our mass.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.10180-b31b1b.svg)](https://arxiv.org/abs/2503.10180) | **Optical+NIR analysis of a newly found Einstein ring at z$\sim$1 from the Kilo-Degree Survey: Dark matter fraction, total and dark matter density slope and IMF**  |
|| R. Li, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *13 pages, 4 figures, 2 tables*|
|**Abstract**|            We report the discovery and spectroscopic confirmation of a new bright blue Einstein ring in the Kilo Degree Survey (KiDS) footprint: the Einstein ``blue eye''. Spectroscopic data from X-Shooter at the Very Large Telescope (VLT) show that the lens is a typical early-type galaxy (ETG) at $z=0.9906$, while the background source is a Ly$\alpha$ emitter at $z=2.823$. The reference lens modeling was performed on a high-resolution $Y-$band adaptive-optics image from HAWK-I at VLT. Assuming a singular isothermal ellipsoid (SIE) total mass density profile, we inferred an Einstein radius $R_{Ein}=10.47 \pm 0.06$ kpc. The average slope of the total mass density inside the Einstein radius, as determined by a joint analysis of lensing and isotropic Jeans equations is $\gamma=2.14^{+0.06}_{-0.07}$, showing no systematic deviation from the slopes of lower redshit galaxies, This can be the evidence of ETGs developing through dry mergers plus moderate dissipationless accretion. Stellar population analysis with 8-band ($gri$ZYJHK$s$) photometries from KiDS and VIKING shows that the total stellar mass of the lens is $M*=(3.95\pm 0.35)\times 10^{11} M_\odot$ (Salpeter Initial Mass Function, IMF), implying a dark matter fraction inside the effective radius to be $f_{\rm DM}=0.307\pm 0.151$. We finally explored the dark matter halo slope and found a strong degeneracy with the dynamic stellar mass. Dark matter adiabatic contraction is needed to explain the posterior distribution of the slope, unless IMF heavier than Salpeter is assumed.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.10346-b31b1b.svg)](https://arxiv.org/abs/2503.10346) | **Model-independent $H_0$ within FLRW: Joint constraints from GWTC-3 standard sirens and strong lensing time delays**  |
|| J.-Y. Song, J.-Z. Qi, J.-F. Zhang, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *7 pages, 1 figure*|
|**Abstract**|            The Hubble tension has emerged as a critical crisis in cosmology, with the cause remaining unclear. Determining the Hubble constant ($H_0$) independently of cosmological models and distance ladders will help resolve this crisis. In this letter, we for the first time use 47 gravitational-wave (GW) standard sirens from the third Gravitational-Wave Transient Catalog to calibrate distances in the strong lensing system, RXJ1131-1231, and constrain $H_0$ through the distance-sum rule, with minimal cosmological assumptions. We assume that light propagation over long distances is described by the Friedmann-Lemaitre-Robertson-Walker metric and that geometrical optics holds, but we do not need to assume the universe's contents or the theory of gravity on cosmological scales. Fixing $\Omega_K=0$, we obtain $H_0=73.22^{+5.95}_{-5.43}$ ${\rm km}~{\rm s}^{-1}~{\rm Mpc}^{-1}$ and $H_0=70.40^{+8.03}_{-5.60}$ ${\rm km}~{\rm s}^{-1}~{\rm Mpc}^{-1}$ by using the deflector galaxy's mass model and kinematic measurements to break mass-sheet transform, respectively. When $\Omega_K$ is not fixed, the central value of $H_0$ increases further. We find that our results are still dominated by statistical errors, and at the same time, we notice the great potential of using GW dark sirens to provide calibration, owing to their higher redshifts. When using 42 binary black holes and RXJ1131-1231, we obtain a $8.46 \%$ $H_0$ constraint precision, which is better than that from the bright siren GW170817 using the Hubble law by about $40\%$. In the future, as the redshift range of GW dark sirens increases, more and more SGLTDs can be included, and we can achieve high-precision, model-independent measurements of $H_0$ without the need for GW bright sirens.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.10423-b31b1b.svg)](https://arxiv.org/abs/2503.10423) | **A search for sterile neutrinos in interacting dark energy models using DESI baryon acoustic oscillations and DES supernovae data**  |
|| L. Feng, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *8 pages, 3 figures*|
|**Abstract**|            Sterile neutrinos can influence the evolution of the universe, and thus cosmological observations can be used to search for sterile neutrinos. In this study, we utilized the latest baryon acoustic oscillations data from DESI, combined with the cosmic microwave background data from Planck and the five-year supernova data from DES, to constrain the interacting dark energy (IDE) models involving both cases of massless and massive sterile neutrinos. We consider four typical forms of the interaction term $Q=\beta H \rho_{\rm de}$, $Q=\beta H \rho_{\rm c}$, $Q=\beta H_{0} \rho_{\rm de}$, and $Q=\beta H_{0} \rho_{\rm c}$, respectively. Our analysis indicates that the current data provide only a hint of the existence of massless sterile neutrinos (as dark radiation) at about the $1\sigma$ level. In contrast, no evidence supports the existence of massive sterile neutrinos. Furthermore, in IDE models, the inclusion of (massless/massive) sterile neutrinos has a negligible impact on the constraint of the coupling parameter $\beta$. The IDE model of $Q=\beta H \rho_{\rm c}$ with sterile neutrinos does not favor an interaction. However, the other three IDE models with sterile neutrinos support an interaction in which dark energy decays into dark matter.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.10035-b31b1b.svg)](https://arxiv.org/abs/2503.10035) | **Temporal variability and obscuration effects in the X-ray emission of classical nova V339 Delphini (Nova Delphini 2013)**  |
|| S. Pei, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-14*|
|*Comments*| *accepted for publication in Monthly Notices of the Royal Astronomical Society (MNRAS), 19 pages, 16 figures,*|
|**Abstract**|            In this study, we present a detailed analysis of public archival soft X-ray data on the classical nova V339 Delphini (Nova Del 2013) during its outburst, obtained using the {\it Chandra} High-Resolution Camera Spectrometer (HRC-S) and Low Energy Transmission Grating (LETG), as well as {\it XMM-Newton} in 2013. The observations, spanning from day 85.2 to day 112.0 after the optical maximum, capture the nova during its luminous supersoft X-ray source (SSS) phase. The spectra reveal numerous absorption features with blue-shifted velocities ranging from $\sim$ 724 to $\sim$ 1474 km s$^{-1}$, with the majority of lines blue-shifted by approximately 1200 km s$^{-1}$. We confirm the presence of a short-period modulation of the X-ray flux with a period of approximately 54 seconds, as well as the drift of this period, which was detected on days 97.0 and 112.0 during the outburst with both {\it XMM-Newton} and {\it Chandra}. This period modulation is transient in nature, with significant variations in amplitude and pulse profile over timescales of a few thousand seconds, likely due to temporary obscuration events that affect the emission from the central hot source. The pulse profiles exhibit substantial deviations from a pure sinusoidal shape, which may be related to the period drift. Additionally, the modulation amplitude shows a possible anti-correlation with the count rates on day 97.0, likely also caused by temporary obscuration events influencing the central source's emission.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.09697/./Figures/SN_50_spectrum.png', 'tmp_2503.09697/./Figures/SN_15_spectrum.png', 'tmp_2503.09697/./Figures/SN_8_spectrum.png', 'tmp_2503.09697/./Figures/FFT_50.png', 'tmp_2503.09697/./Figures/FFT_15.png', 'tmp_2503.09697/./Figures/FFT_8.png', 'tmp_2503.09697/./Figures/FFT_Panel_2.png', 'tmp_2503.09697/./Figures/MeanDiff_vs_SNR.png']
copying  tmp_2503.09697/./Figures/SN_50_spectrum.png to _build/html/
copying  tmp_2503.09697/./Figures/SN_15_spectrum.png to _build/html/
copying  tmp_2503.09697/./Figures/SN_8_spectrum.png to _build/html/
copying  tmp_2503.09697/./Figures/FFT_50.png to _build/html/
copying  tmp_2503.09697/./Figures/FFT_15.png to _build/html/
copying  tmp_2503.09697/./Figures/FFT_8.png to _build/html/
copying  tmp_2503.09697/./Figures/FFT_Panel_2.png to _build/html/
copying  tmp_2503.09697/./Figures/MeanDiff_vs_SNR.png to _build/html/
exported in  _build/html/2503.09697.md
    + _build/html/tmp_2503.09697/./Figures/SN_50_spectrum.png
    + _build

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\orcid}[1]{\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt]{orcid.png}}}$</div>



<div id="title">

# $_ WI_$ggle $_ C_$orrector $_ K_$it for NIRSp$_ E_$c $_ D_$ata: WICKED

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.09697-b31b1b.svg)](https://arxiv.org/abs/2503.09697)<mark>Appeared on: 2025-03-14</mark> -  _Submitted to A&A_

</div>
<div id="authors">

<mark>A. Dumont</mark>, et al. -- incl., <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** $_ Context:_$ The point-spread function of the integral-field unit (IFU) mode of the Near-Infrared Spectrograph (NIRSpec) detector of JWST is heavily under-sampled. The resampling of the spectra into a 3D data cube creates resampling noise seen as low-frequency sinusoidal-like artifacts, or "wiggles". These artifacts in the data are not corrected in the JWST data pipeline, and significantly impact the science that can be achieved at a single-pixel level. $_ Aims:_$ Here we present the tool " $_ WI_$ ggle $_ C_$ orrector $_ K_$ it for NIRSp $_ E_$ c $_ D_$ ata" ( ${\sc WICKED}$ ), designed to remove these artifacts. While fully characterizing wiggles requires forward modeling of the instrument response, ${\sc WICKED}$ offers a faster, computationally efficient alternative using an empirical correction. $_ Methods:_$ ${\sc WICKED}$ uses the  Fast Fourier Transform to identify wiggle-affected spaxels across the (IFU) data cube. Spectra are modeled with a mix of integrated aperture and annular templates, a power-law, and a second-degree polynomial, avoiding high-degree polynomials that distort spectral features. Our correction works across all medium and high-resolution NIRSpec gratings: F070LP, F100LP, F170LP, and F290LP. $_ Results:_$ ${ \sc WICKED}$ can recover the true overall spectral shape up to a factor of 3.5 $\times$ better compared to uncorrected spectra. It recovers the equivalent width of  absorption lines within 5 \% of the true value \textemdash $\sim$ 3 $\times$ better than uncorrected spectra and $\sim2\times$ better than other methods. ${\sc WICKED}$ significantly improves kinematic measurements, recovering the line-of-sight velocity (LOSV) within 1 \% of the true value \textemdash more than $100\times$ better than uncorrected spectra at S/N $\sim$ 40 . The superior wiggle-removal capabilities of ${\sc WICKED}$ also reduces the LOSV uncertainties by $\sim$ 50 \% compared to other methods. As a case study, we applied ${\sc WICKED}$ to G235H/F170LP IFU data of the elliptical galaxy NGC 5128, finding good agreement with previous studies. In wiggle-affected regions, the uncorrected spectrum showed stellar LOSV and velocity dispersion differences compared to the ${\sc WICKED}$ -cleaned spectrum, of $\sim$ 17 $\times$ and $\sim$ 36 $\times$ larger than the estimated uncertainties, respectively. $_ Conclusions:_$ ${Wiggles in NIRSpec IFU data can significantly distort the overall spectral shape, bias line measurements and kinematics to values larger than the expected uncertainties for uncorrected spectra. \sc WICKED}$ is a robust, user-friendly solution for mitigating wiggles in NIRSpec data. Unlike other methods, it minimizes residual artifacts, enabling precise single-pixel studies, enhancing JWST’s potential for groundbreaking discoveries in galaxy kinematics and early universe studies.

</div>

<div id="div_fig1">

<img src="tmp_2503.09697/./Figures/SN_50_spectrum.png" alt="Fig3.1" width="16%"/><img src="tmp_2503.09697/./Figures/SN_15_spectrum.png" alt="Fig3.2" width="16%"/><img src="tmp_2503.09697/./Figures/SN_8_spectrum.png" alt="Fig3.3" width="16%"/><img src="tmp_2503.09697/./Figures/FFT_50.png" alt="Fig3.4" width="16%"/><img src="tmp_2503.09697/./Figures/FFT_15.png" alt="Fig3.5" width="16%"/><img src="tmp_2503.09697/./Figures/FFT_8.png" alt="Fig3.6" width="16%"/>

**Figure 3. -** Comparison of the spectrum of the A-star J1757132 (top, black) and the degraded spectrum (top, red) with the added wiggle model (green) at _ S/N_ ratios of $50$, $15$ and $8$.
_ Middle:_ the Fourier spectra of i) the degraded spectrum (solid, red), ii) the wiggle model (green), iii) the degraded spectrum without wiggles (black), and iv) the data corrected with {\sc WICKED}(dashed, red).
The horizontal lines mark the mean amplitude of the Fourier spectrum at frequencies dominated by wiggles and at larger frequencies. The Fourier ratio can effectively distinguish wiggles from noise down to a _ S/N_ ratio of $\sim 8$. _ Bottom:_ comparison of the input wiggle model (green) versus the recovered wiggle spectrum for the data cleaned with {\sc WICKED}. (*fig:FFT_test_SNR*)

</div>
<div id="div_fig2">

<img src="tmp_2503.09697/./Figures/FFT_Panel_2.png" alt="Fig2" width="100%"/>

**Figure 2. -** The Fast Fourier Transform is used in {\sc WICKED} to flag spaxels in the data cube affected by wiggles. The built-in method {\sc plot\_wiggle\_FFT} in WICKED allows for manual examination of the spectrum and its Fourier transform for a specific spaxel in the datacube.
    Left panel shows the wiggle spectrum, created by subtracting the spectrum from the best-fit model. Right panel shows the Fourier transform of the wiggle spectrum. The solid fuchsia line represents the mean amplitude in the wiggle-dominated part of the spectrum (shaded red region). WICKED flags spaxels by comparing this value to the standard deviation (blue dashed line). If data from both NRS detectors are available, as in this case, WICKED determines which part of the spectrum shows the most prominent wiggles and bases the flagging on that part of the spectrum.
     (*fig:FFT-example*)

</div>
<div id="div_fig3">

<img src="tmp_2503.09697/./Figures/MeanDiff_vs_SNR.png" alt="Fig5" width="100%"/>

**Figure 5. -** Comparison of the mean difference flux respect to the aperture spectrum of the A-star J175713. The black rectangles represent the minimum expected the flux difference at a given _ S/N_ ratio. The spectra corrected using {\sc WICKED}(solid red) shows the smallest mean flux difference compared to the uncorrected spectrum (gray) and the spectrum clean using the method by [Perna, Arribas and Marshall (2023)]()(solid yellow) at all _ S/N_.   (*fig:MeanDiff_vs_SNR*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.09697"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# $\large{Low 4.5$ \mu$m  Dayside Emission Disfavors a Dark Bare-Rock scenario\ for the Hot Super-Earth TOI-431 b}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.09698-b31b1b.svg)](https://arxiv.org/abs/2503.09698)<mark>Appeared on: 2025-03-14</mark> -  _14 pages, 5 figures, accepted for publication in The Astronomical Journal_

</div>
<div id="authors">

C. Monaghan, et al. -- incl., <mark>I. J. M. Crossfield</mark>, <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** The full range of conditions under which rocky planets can host atmospheres remains poorly understood, especially in the regime of close-in orbits around late-type stars. One way to assess the presence of atmospheres on rocky exoplanets is to measure their dayside emission as they are eclipsed by their host stars. Here, we present Spitzer observations of the 4.5 µm secondary eclipses of the rocky super-Earth TOI-431 b, whose mass and radius indicate an Earth-like bulk composition (3.07 $\pm 0.35$ M $_{\oplus}$ , 1.28 $\pm 0.04$ R $_{\oplus}$ ). Exposed to more than 2000 times the irradiation of Earth, dayside temperatures of up to 2400 K are expected if the planet is a dark bare-rock without a significant atmosphere. Intriguingly, despite the strong stellar insolation, we measure a secondary eclipse depth of only 33 $\pm$ 22 ppm, which corresponds to a dayside brightness temperature of $1520_{-390}^{+360}$ K. This notably low eclipse depth disagrees with the dark bare-rock scenario at the 2.5 $\sigma$ level, and suggests either that the planet is surrounded by an atmosphere, or that it is a bare-rock with a highly reflective surface. In the atmosphere scenario, the low dayside emission implies the efficient redistribution of heat to the nightside, or by molecular absorption in the 4--5 $\mu$ m bandpass. In the bare-rock scenario, a surface composition made of a high-albedo mineral species such as ultramafic rock can lead to reduced thermal emission consistent with low eclipse depth measurement. Follow-up spectroscopic observations with the James Webb Space Telescope hold the key to constraining the nature of the planet.

</div>

<div id="div_fig1">

<img src="tmp_2503.09698/./scarlet_models_andopacity.png" alt="Fig5" width="100%"/>

**Figure 5. -** Eclipse depth of TOI-431 b at 4.5 µm compared to a suite of theoretical emission spectra produced by SCARLET. The coloured lines represent the five atmospheric models simulated by SCARLET using a non-gray temperature profile and a well-mixed composition. The dashed gray lines show the blackbody eclipse spectra corresponding to different combinations of $f$ and A$_{B}$ at uniform dayside temperatures. The black dot represents the measured eclipse depth, with the associated $\pm1\sigma$ error bars. Coloured circles represent the models integrated over the IRAC2 bandpass between 4.05 and 4.95  µm. The exact compositions of the atmospheric models are shown in Table \ref{tab:models}. Normalized opacity bands from dominating molecules in the models (CO, CO$_{2}$, and SiO) are shown in the bottom panel. (*fig:atmospheremodels*)

</div>
<div id="div_fig2">

<img src="tmp_2503.09698/./sigmagrid.png" alt="Fig3" width="100%"/>

**Figure 3. -** Joint posterior distribution of the Bond albedo A$_{B}$(vertical axis) and heat redistribution factor $f$(horizontal axis) assuming TOI-431 b radiates as blackbody at its 4.5 µm brightness temperature of $1520$K. The 2D posterior is shown using colored shading, where darker regions indicate lower probabilities. The 1$\sigma$ and 2$\sigma$ regions are indicated by the black contour lines. The approximate values for the Moon and Mercury are shown in yellow and magenta at the bottom right, respectively. (*fig:blackbodyAbF*)

</div>
<div id="div_fig3">

<img src="tmp_2503.09698/./eclipsecomparison.png" alt="Fig1" width="100%"/>

**Figure 1. -** Measured eclipse depths for all seven visits using three different systematic models, including the associated error bars of each. The rightmost points show the weighted average across all visits for the individual systematics. (*fig:systematicscomparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.09698"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\myslash}{\operatorname{/}}$
$\newcommand{\myleftbracket}{\left[}$
$\newcommand{\myrightbracket}{\right]}$
$\newcommand{\highlight}[1]{\textcolor{black}{\textnormal{{#1}}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\teff}{\mbox{T_{\text{eff}}}}$
$\newcommand{\msun}{\mbox{M_{\odot}}}$
$\newcommand{\logg}{\mbox{\log g}}$
$\newcommand{\feh}{\mbox{[Fe/H]}}$
$\newcommand{\vsini}{\mbox{v \text{sin} i}}$
$\newcommand{\logrhk}{\mbox{\log R^{\prime}_{\text{HK}}}}$
$\newcommand{\alpham}{\mbox{[\rm \alpha/M]}}$
$\newcommand{\yar}{\mbox{young \alpha--rich}}$
$\newcommand{\yap}{\mbox{young \alpha--poor}}$
$\newcommand{\oar}{\mbox{old \alpha--rich}}$
$\newcommand{\halpha}{\mbox{H_{\alpha}}}$
$\newcommand{\primerhk}{\mbox{R^{\prime}_{\textrm{HK}}}}$
$\newcommand{\rhk}{\mbox{R_{\textrm{HK}}}}$
$\newcommand{\rhkphot}{\mbox{R_{\textrm{HK,phot}}}}$
$\newcommand{\dwirt}{\mbox{\Delta W_{\textrm{IRT}}}}$
$\newcommand{\dsindex}{\mbox{\Delta S_{\textrm{HK}}}}$
$\newcommand{\dfeh}{\mbox{\Delta[\textrm{Fe/H]}}}$
$\newcommand{\sigmafeh}{\mbox{\sigma_{[\textrm{Fe/H]}}}}$
$\newcommand{\sindex}{\mbox{S_{\textrm{HK}}}}$
$\newcommand{\dxh}{\mbox{\Delta[\textit{X}/\textrm{H}]}}$
$\newcommand{\dxo}{\mbox{\Delta[\textit{X}/\textrm{O}]}}$
$\newcommand{\dxhdtcont}{\frac{\partial \Delta \myleftbracket X \myslash \rm{H} \myrightbracket}{\partial T_{\rm{cond}}}}$
$\newcommand{\dxhdwirt}{\frac{\partial \Delta \myleftbracket X \myslash \rm{H} \myrightbracket}{\partial \Delta W_{\textrm{IRT}}}}$
$\newcommand$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# $\centering{C3PO IV: co-natal stars depleted in refractories are magnetically more active--- possible imprints of planets}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.10339-b31b1b.svg)](https://arxiv.org/abs/2503.10339)<mark>Appeared on: 2025-03-14</mark> -  _13 pages, 12 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

J. Yu, et al. -- incl., <mark>J. Liu</mark>

</div>
<div id="abstract">

**Abstract:** Chemical abundance anomalies in twin stars have recently been considered tell-tale signs of interactions between stars and planets. While such signals are prevalent, their nature remains a subject of debate. On one hand, exoplanet formation may induce chemical depletion in host stars by locking up refractory elements. On the other hand, exoplanet engulfment can result in chemical enrichment, both processes potentially producing similar differential signals. In this study, we aim to observationally disentangle these processes by using the Ca II infrared triplet to measure the magnetic activity of 125 co-moving star pairs with high SNR, high-resolution spectra from the Magellan, Keck, and VLT telescopes. We find that co-natal star pairs in which the two stars exhibit significant chemical abundance differences also show differences in their magnetic activity, with stars depleted in refractories being magnetically more active. Furthermore, the strength of this correlation between differential chemical abundances and differential magnetic activity increases with condensation temperature. One possible explanation is that the chemical anomaly signature may be linked to planet formation, wherein refractory elements are locked into planets, and the host stars become more active due to more efficient contraction during the pre-main-sequence phase or star–planet tidal and magnetic interactions.

</div>

<div id="div_fig1">

<img src="tmp_2503.10339/./Activity-EWdiffMean-ChemDiff-delta-xh-C-twoGroup.png" alt="Fig5.1" width="25%"/><img src="tmp_2503.10339/./Activity-EWdiffMean-ChemDiff-delta-xh-Na-twoGroup.png" alt="Fig5.2" width="25%"/><img src="tmp_2503.10339/./Activity-EWdiffMean-ChemDiff-delta-xh-Fe2-twoGroup.png" alt="Fig5.3" width="25%"/><img src="tmp_2503.10339/./Activity-EWdiffMean-ChemDiff-delta-xh-Sc2-twoGroup.png" alt="Fig5.4" width="25%"/>

**Figure 5. -** \bc{Relationships between the differential activity index, $\dwirt$, and the elemental abundance difference, $\Delta$[X/H], for Carbon (**a**), Sodium (**b**), Iron II (**c**), and Scandium II (**d**). The condensation temperatures of these elements increase from 40 K for C to 958 K for Na, 1334 K for Fe II, and 1659 K for Sc II, as noted in each panel. Red circles indicate star pairs with chemical anomalies, while grey circles represent those without. The red dashed line shows the best linear fit to the $\dwirt$–$\Delta$[X/H] relation for the chemical anomalies sample, whereas the black dashed line represents the fit for the chemical homogeneous sample. Shaded regions indicate 1$\sigma$, 2$\sigma$, and 3$\sigma$ uncertainties, with blue for the chemical anomalies sample and grey for the homogeneous sample (see legend in panel a). The slope of the red dashed line and its 1$\sigma$ uncertainty are indicated in each panel. } (*fig:ActivityAbundance*)

</div>
<div id="div_fig2">

<img src="tmp_2503.10339/./Activity-EWdiffMean-ChemDiff-CondTeff.png" alt="Fig9" width="100%"/>

**Figure 9. -** Correlation between condensation temperature and the slope of the $\dwirt$–$\dxh$ relation (i.e., $\dxh$dwirt, with 4 examples illustrated in Fig. \ref{fig:ActivityAbundance}), where _X_ represents 21 individual elements annotated in the plot. Co-natal systems with chemical anomalies (i.e., $|\dfeh| \geq 3\sigma_{\dfeh}$) are represented by red circles, while those without chemical anomalies (i.e., $|\dfeh| < 3\sigma_{\dfeh}$) are represented by black squares (see legend). The red and black solid lines represent the best linear fits for the chemically anomalous and homogeneous groups, respectively, considering only refractory elements with condensation temperatures greater than $900 \mathrm{K}$. These lines are extrapolated to lower condensation temperatures ($<900 \mathrm{K}$) and are shown as dashed lines in this range. The blue and grey shaded regions represent the 1--$\sigma$ uncertainties of the fits.  The slope of the red dashed line is $\left(-1.89\pm0.13\right) \times 10^{-5}$, while the slope of the black dashed line is $\left(2.54\pm1.19\right) \times 10^{-6}$. Including volatile elements in the fitting slightly alters the slopes and  intercepts. (*fig:CondenTeff*)

</div>
<div id="div_fig3">

<img src="tmp_2503.10339/./ChemicalPatternsSlopeEWdiff.png" alt="Fig4" width="100%"/>

**Figure 4. -** Differential activity index ($\dwirt$) as a function of the slope of the condensation temperature versus elemental abundance difference relation ($\dxh$dtcont) for 21 co-natal systems exhibiting chemical anomalies (i.e., $|\dfeh| \geq 3\sigma_{\dfeh}$). The red dashed line represents the best fit obtained through weighted linear regression, minimizing the weighted sum of squared residuals between observed data and the fitted line, with weights as the inverse square of the uncertainty in $\dxh$dtcont as determined from the linear fit. The shaded purple regions indicate the 1-, 2-, and 3-$\sigma$ uncertainties around the best fit, with the slope value annotated. Green arrows highlight four pairs suggested by liu2024 as potential exoplanet engulfment instances, with corresponding pair IDs. (*fig:chemistrytcondactivity*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.10339"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

364  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
